# **Authors:** 
### **- Héctor A. Robles P.**
###  **- Gregory G. Richarte P.**
###  **- Brandon S. Martinez L.**

In [4]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from selenium import webdriver
import chromedriver_autoinstaller
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
from joblib import Parallel, delayed 
from concurrent import futures

In [5]:
def get_datos(matriz, y_test, precision):
     # Definición de datos del modelo
    datos = {
        'Precision Global': [],
        'Error Global': [],
        'Precision Positiva': [],
        'Precision Negativa': [],
        'Falsos Positivos (FP)': [],
        'Falsos Negativos(FN)': [],
        'Asertividad Positiva(AP)': [],
        'Asertividad Negativa(AN)': [],
        'Precision del modelo': []
    }
    
    # Almacenamiento de los valores en el diccionario datos
    datos['Precision Global'].append((matriz[0, 0] + matriz[1, 1]) / len(y_test))
    datos['Error Global'].append((matriz[0, 1] + matriz[1, 0]) / len(y_test))
    datos['Precision Positiva'].append(matriz[1, 1] / (matriz[1, 1] + matriz[0, 1]))
    datos['Precision Negativa'].append(matriz[0, 0] / (matriz[0, 0] + matriz[1, 0]))
    datos['Falsos Positivos (FP)'].append(matriz[0, 1] / (matriz[0, 1] + matriz[0, 0]))
    datos['Falsos Negativos(FN)'].append(matriz[1, 0] / (matriz[1, 0] + matriz[1, 1]))
    datos['Asertividad Positiva(AP)'].append(matriz[1, 1] / (matriz[1, 1] + matriz[0, 1]))
    datos['Asertividad Negativa(AN)'].append(matriz[0, 0] / (matriz[0, 0] + matriz[1, 0]))
    datos['Precision del modelo'].append(precision)
    
    return datos

In [6]:
#Cargar pagina
movie_list = [
  "https://www.imdb.com/title/tt0149460/reviews",   # Futurama
  "https://www.imdb.com/title/tt0386676/reviews",   # The Office
  "https://www.imdb.com/title/tt0903747/reviews"   # Breaking Bad
  "https://www.imdb.com/title/tt1520211/reviews",   # The Walking Dead
  "https://www.imdb.com/title/tt4574334/reviews",   # Stranger Things
  "https://www.imdb.com/title/tt1190634/reviews"   # The Boys
  "https://www.imdb.com/title/tt0412142/reviews",   # House M.D.
  "https://www.imdb.com/title/tt0182576/reviews",   # Family Guy
  "https://www.imdb.com/title/tt0068646/reviews",   # The Godfather
  "https://www.imdb.com/title/tt2861424/reviews",   # Rick and Morty
  "https://www.imdb.com/title/tt0121955/reviews",   # South Park
  "https://www.imdb.com/title/tt2521884/reviews",   # Drawn Together
  "https://www.imdb.com/title/tt0898266/reviews",   # The Big Bang Theory
  "https://www.imdb.com/title/tt2085059/reviews",   # Black Mirror
  "https://www.imdb.com/title/tt0098844/reviews",   # Law & Order
  "https://www.imdb.com/title/tt0212671/reviews",   # Malcolm in the Middle
  "https://www.imdb.com/title/tt0460649/reviews",   # How I Met Your Mother
  "https://www.imdb.com/title/tt7366338/reviews",   # Chernobyl
  "https://www.imdb.com/title/tt0141842/reviews",   # The Sopranos
  "https://www.imdb.com/title/tt1099212/reviews",   # Dragonball Evolution
  "https://www.imdb.com/title/tt0413573/reviews",   # Grey's Anatomy
  "https://www.imdb.com/title/tt1506229/reviews",   # The Beatles: Get Back
  "https://www.imdb.com/title/tt1865718/reviews",   # Gravity Falls
  "https://www.imdb.com/title/tt0096697/reviews",   # The Simpsons
  "https://www.imdb.com/title/tt0285403/reviews" ,  # The Last of Us
  "https://www.imdb.com/title/tt0068468/reviews"    # El Chavo del Ocho
]

def get_serie(movie):
    data = {
        'comment':[],
        'score': [],
        'rate': []
    }

    chromedriver_autoinstaller.install()
    driver = webdriver.Chrome()

    driver.get(movie)
    time.sleep(2)
    for _ in range(20):
        try:
            load_more_button = WebDriverWait(driver, 10).until(
                EC.visibility_of_element_located((By.ID, "load-more-trigger"))
            )

            load_more_button.click()

            time.sleep(2)
        except:
            break

    updated_html_content = driver.page_source

    body = BeautifulSoup(updated_html_content, 'html.parser')

    opinions = body.find_all('div', class_='lister-item')

    for o in opinions:
        try:
            score = int(list(o.find('span', class_='rating-other-user-rating').children)[3].text)
            comment = o.find('div', class_='text').text
        except:
            continue

        data['comment'].append(comment)
        data['score'].append(score)
        if(score >= 7):
            data['rate'].append('1')
        else:
            data['rate'].append('0')

    driver.quit()

    return data

# df = pd.DataFrame(data)

with futures.ThreadPoolExecutor() as executor: # default/optimized number of threads
  titles = list(executor.map(get_serie, movie_list))

In [7]:
all_data = {
    'comment':[],
    'score': [],
    'rate': []
}

for t in titles:
    all_data['comment'] += t['comment']
    all_data['score'] += t['score']
    all_data['rate'] += t['rate']

df = pd.DataFrame(all_data)
df
data1 = df

#Los datos son usados para CSV para su procesamiento

In [8]:
# csv_data = df.to_csv('dataset_1.csv', index = False)


# PNL

In [9]:
#Libraries
import matplotlib.pyplot as plt
import os

from sklearn.preprocessing import LabelEncoder
from sklearn.utils import resample
from sklearn.feature_extraction.text import CountVectorizer
from nltk.sentiment.vader import SentimentIntensityAnalyzer

from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.stem import PorterStemmer, LancasterStemmer
from nltk.stem.snowball import SnowballStemmer
from nltk.corpus import stopwords
from nltk.corpus import wordnet
import string
from string import punctuation
import nltk
import re

In [10]:
nltk.download('stopwords')
nltk.download('vader_lexicon')
sentiments = SentimentIntensityAnalyzer()
df["Positive"] = [sentiments.polarity_scores(i)["pos"] for i in df['comment']]
df["Negative"] = [sentiments.polarity_scores(i)["neg"] for i in df['comment']]
# df["Neutral"] = [sentiments.polarity_scores(i)["neu"] for i in df['comment']]
df["Compound"] = [sentiments.polarity_scores(i)["compound"] for i in df['comment']]
score = df["Compound"].values
sentiment = []
for i in score:
    if i >= 0.05:
        sentiment.append('Positive')
    elif i < 0.05:
        sentiment.append('Negative')
    # else:
    #     sentiment.append('Neutral')

df["Sentiment"] = sentiment
df.head()

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/scavenger/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package vader_lexicon to
[nltk_data]     /home/scavenger/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


,comment,score,rate,Positive,Negative,Compound,Sentiment
0,"This show, well lets say it's probably one of ...",8,1,0.201,0.012,0.9786,Positive
1,I was so excited when I heard about new season...,9,1,0.106,0.202,-0.9848,Negative
2,The television series Futurama is nothing shor...,8,1,0.131,0.051,0.9829,Positive
3,Now that is not to say the Simpsons is bad or ...,10,1,0.414,0.021,0.9990,Positive
4,This show is so brilliant you don't deserve to...,10,1,0.327,0.000,0.7065,Positive


In [11]:
data2 = df.drop(['Positive', 'Negative', 'Compound'], axis=1)
data2.head()

,comment,score,rate,Sentiment
0,"This show, well lets say it's probably one of ...",8,1,Positive
1,I was so excited when I heard about new season...,9,1,Negative
2,The television series Futurama is nothing shor...,8,1,Positive
3,Now that is not to say the Simpsons is bad or ...,10,1,Positive
4,This show is so brilliant you don't deserve to...,10,1,Positive


In [12]:
stop_words = stopwords.words('english')
porter_stemmer = PorterStemmer()
lancaster_stemmer = LancasterStemmer() 
snowball_stemer = SnowballStemmer(language="english")
lzr = WordNetLemmatizer()

In [13]:
def text_processing(text):   
    # convert text into lowercase
    text = text.lower()

    # remove new line characters in text
    text = re.sub(r'\n',' ', text)
    
    # remove punctuations from text
    text = re.sub('[%s]' % re.escape(punctuation), "", text)
    
    # remove references and hashtags from text
    text = re.sub("^a-zA-Z0-9$,.", "", text)
    
    # remove multiple spaces from text
    text = re.sub(r'\s+', ' ', text, flags=re.I)
    
    # remove special characters from text
    text = re.sub(r'\W', ' ', text)

    text = ' '.join([word for word in word_tokenize(text) if word not in stop_words])
    
    # lemmatizer using WordNetLemmatizer from nltk package
    text=' '.join([lzr.lemmatize(word) for word in word_tokenize(text)])

    return text

In [14]:
nltk.download('omw-1.4')
data_copy = data2.copy()
data_copy.comment = data_copy.comment.apply(lambda text: text_processing(text))

[nltk_data] Downloading package omw-1.4 to
[nltk_data]     /home/scavenger/nltk_data...


In [15]:
le = LabelEncoder()
data_copy['Sentiment'] = le.fit_transform(data_copy['Sentiment'])

In [16]:
processed_data = {
    'Sentence':data_copy.comment,
    'Sentiment':data_copy['Sentiment']
}

processed_data = pd.DataFrame(processed_data)
processed_data.head()

,Sentence,Sentiment
0,show well let say probably one original show s...,1
1,excited heard new season watched watched next ...,0
2,television series futurama nothing short brill...,1
3,say simpson bad anything really like jumped sh...,1
4,show brilliant dont deserve watch,1


In [17]:
processed_data['Sentiment'].value_counts()

Sentiment
1    5723
0    1336
Name: count, dtype: int64

In [18]:
df_negative = processed_data[(processed_data['Sentiment']==0)]
df_positive = processed_data[(processed_data['Sentiment']==1)]

final_data = pd.concat([df_negative, df_positive])

In [19]:
#Funcion para convertir en CSV
csv_data = final_data.to_csv('dataset_1.csv', index = False)

In [20]:
final_data['Sentiment'].value_counts()

Sentiment
1    5723
0    1336
Name: count, dtype: int64

In [21]:
corpus = []
for sentence in final_data['Sentence']:
    corpus.append(sentence)
corpus[0:5]

['excited heard new season watched watched next episode next fourth last one watch know writing bad stop ruin whole show new material like bad ripoff original show still dvd occasionally watchi excited heard new season watched watched next episode next fourth last one watch know writing bad stop ruin whole show new material like bad ripoff original show still dvd occasionally watchi excited heard new season watched watched next episode next fourth last one watch know writing bad stop ruin whole show new material like bad ripoff original show still dvd occasionally watch',
 'first three season really interesting funny long time laugh many episode however fourth sixth everything changed joke became somehow flat banal toilet humor overly stupid futuramastyle absurdly funny style simply stupid like episode robot took gun stupidly shoot bullet air half minute episode complains gun empty useless joke happened joke sake joke rudely thrown episode hope viewer take style series result season co

In [22]:
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(max_features=1500)
X = cv.fit_transform(corpus).toarray()
y = final_data.iloc[:, -1].values

In [23]:
X

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 1, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

## Modelo Navie Bayes

In [24]:
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import confusion_matrix, precision_score
from sklearn.model_selection import train_test_split


# Definir X_train, X_test, y_train, y_test usando train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0)

# Definición del algoritmo Naive Bayes
algoritmo = GaussianNB()

# Entrenamiento del modelo
algoritmo.fit(X_train, y_train)

# Realización de la predicción
y_pred = algoritmo.predict(X_test)

# Cálculo de la matriz de confusión
matriz = confusion_matrix(y_test, y_pred)
print('Matriz de Confusion:')
print(matriz)

# Cálculo de la precisión del modelo
nb_precision = precision_score(y_test, y_pred)
print('Precision del modelo:')
print(nb_precision)

# Convertir el diccionario en un DataFrame
nb_datos = pd.DataFrame(get_datos(matriz, y_test, nb_precision))
indice_1 = pd.DataFrame(get_datos(matriz, y_test, nb_precision))

Matriz de Confusion:
[[295 102]
 [781 940]]
Precision del modelo:
0.9021113243761996


## Modelo NNBP

In [25]:
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import confusion_matrix, precision_score
from sklearn.model_selection import train_test_split

# Definir X_train, X_test, y_train, y_test usando train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0)

# Definición del algoritmo MLP
algoritmo = MLPClassifier()

# Entrenamiento del modelo
algoritmo.fit(X_train, y_train)

# Realización de la predicción
y_pred = algoritmo.predict(X_test)

# Cálculo de la matriz de confusión
matriz = confusion_matrix(y_test, y_pred)
print('Matriz de Confusion:')
print(matriz)

# Cálculo de la precisión del modelo
mlp_precision = precision_score(y_test, y_pred)
print('Precision del modelo:')
print(mlp_precision)

# Convertir el diccionario en un DataFrame

mlp_datos = pd.DataFrame(get_datos(matriz, y_test, mlp_precision))
indice_2 = pd.DataFrame(get_datos(matriz, y_test, mlp_precision))

Matriz de Confusion:
[[ 224  173]
 [ 119 1602]]
Precision del modelo:
0.9025352112676056


##  MAQUINAS DE VECTORES DE SOPORTE

In [26]:
from sklearn.svm import SVC
from sklearn.metrics import confusion_matrix, precision_score

# Definición del algoritmo SVM
algoritmo = SVC(kernel='linear')

# Entrenamiento del modelo
algoritmo.fit(X_train, y_train)

# Realización de la predicción
y_pred = algoritmo.predict(X_test)

# Cálculo de la matriz de confusión
matriz = confusion_matrix(y_test, y_pred)
print('Matriz de Confusion:')
print(matriz)

# Cálculo de la precisión del modelo
svm_precision = precision_score(y_test, y_pred)
print('Precision del modelo:')
print(svm_precision)

# Convertir el diccionario en un DataFrame
svm_datos = pd.DataFrame(get_datos(matriz, y_test, svm_precision))
indice_3 = pd.DataFrame(get_datos(matriz, y_test, svm_precision))

Matriz de Confusion:
[[ 222  175]
 [ 163 1558]]
Precision del modelo:
0.8990190421234853


## Tabla de comparacion

In [27]:
# Convertir los datos en DataFrames
indice_datos = {
    'Precision Positiva': [nb_datos['Precision Positiva'][0], mlp_datos['Precision Positiva'][0], svm_datos['Precision Positiva'][0]],
    'Precision Negativa': [nb_datos['Precision Negativa'][0], mlp_datos['Precision Negativa'][0], svm_datos['Precision Negativa'][0]],
    'Precision Global': [nb_datos['Precision Global'][0], mlp_datos['Precision Global'][0], svm_datos['Precision Global'][0]],
    'Error Global': [nb_datos['Error Global'][0], mlp_datos['Error Global'][0], svm_datos['Error Global'][0]],
    'Falsos Positivos (FP)': [nb_datos['Falsos Positivos (FP)'][0], mlp_datos['Falsos Positivos (FP)'][0], svm_datos['Falsos Positivos (FP)'][0]],
    'Falsos Negativos(FN)': [nb_datos['Falsos Negativos(FN)'][0], mlp_datos['Falsos Negativos(FN)'][0], svm_datos['Falsos Negativos(FN)'][0]],
    'Asertividad Positiva(AP)': [nb_datos['Asertividad Positiva(AP)'][0], mlp_datos['Asertividad Positiva(AP)'][0], svm_datos['Asertividad Positiva(AP)'][0]],
    'Asertividad Negativa(AN)': [nb_datos['Asertividad Negativa(AN)'][0], mlp_datos['Asertividad Negativa(AN)'][0], svm_datos['Asertividad Negativa(AN)'][0]],
    'Precision del modelo': [nb_precision, mlp_precision, svm_precision]
}

indice_ids = ['Naive Bayes', 'MLP', 'SVM']
indice_df = pd.DataFrame(indice_datos, index=indice_ids)

print(indice_df)


             Precision Positiva  Precision Negativa  Precision Global  \
Naive Bayes            0.902111            0.274164          0.583097   
MLP                    0.902535            0.653061          0.862134   
SVM                    0.899019            0.576623          0.840415   

             Error Global  Falsos Positivos (FP)  Falsos Negativos(FN)  \
Naive Bayes      0.416903               0.256927              0.453806   
MLP              0.137866               0.435768              0.069146   
SVM              0.159585               0.440806              0.094712   

             Asertividad Positiva(AP)  Asertividad Negativa(AN)  \
Naive Bayes                  0.902111                  0.274164   
MLP                          0.902535                  0.653061   
SVM                          0.899019                  0.576623   

             Precision del modelo  
Naive Bayes              0.902111  
MLP                      0.902535  
SVM                      0.899019

## Concluciones.
### Bibliografia.
### URL: SCIKIT-LEARN (Machine Learning in Python 2022). https://scikit-learn.org/stable/¶